# CAPP 30239 - Assignment 2 : Static Viz 1 (Intro & Exploratory)

*Chicago Public Schools (SY 2018-2019) vs. Chicago Crimes*

LIZA MARIE SORIANO


## Setup

In [0]:
import sys
import pandas as pd
import json
import pandas as pd
import altair as alt
#alt.renderers.enable('colab')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Data Used
- [Chicago Public Schools Profile Information for Schoolyear 2018-2019](https://data.cityofchicago.org/Education/Chicago-Public-Schools-School-Profile-Information-/kh4r-387c)
- [Chicago Public Schools Graduation Outcomes 2011-2019](https://cps.edu/SchoolData/Pages/SchoolData.aspx)
- [Chicago Crimes Data for Year 2018](https://data.cityofchicago.org/Public-Safety/Crimes-2018/3i3m-jwuy)

In [0]:
CPS_1819 = "Chicago_Public_Schools_-_School_Profile_Information_SY1819"
CPS_Metrics = "Metrics_CohortGraduationDropoutAdjusted_SchoolLevel_2011to2019"
CRIMES_18 = "Crimes_-_2018"

## Helper Functions

In [0]:
def path_to_data(file):
  return '/content/drive/My Drive/Courses/CAPP 30239 - Data Viz for Policy Analysis/Data/{}'.format(file)

In [0]:
def read_csv(filename, cols=None):
  '''
  Inputs:
    name of file (str)--must end in .csv
    columns to include from original (list)--if not specified, read all cols
  Outpts:
  '''
  if not cols:
    df = pd.read_csv(path_to_data(filename))
  else:
    df = pd.read_csv(path_to_data(filename), usecols=cols)
  return df

## CPS DATA

### Read & Modify Data

In [0]:
cps_df = read_csv(CPS_1819 + ".csv")

In [0]:
''' ADD ATTRIBUTES '''
# Add School Year
cps_df['schoolyear'] = '2018-2019'

# Add % Demographics
dmg_perc_cols = {'Student_Count_Low_Income': '%low_inc', \
                 'Student_Count_Special_Ed': '%special_ed', \
                 'Student_Count_English_Learners': '%esl', \
                 'Student_Count_Black': '%black', \
                 'Student_Count_Hispanic': '%hisp', \
                 'Student_Count_White': '%white', \
                 'Student_Count_Asian': '%asian', \
                 'Student_Count_Native_American': '%native', \
                 'Student_Count_Other_Ethnicity': '%other', \
                 'Student_Count_Asian_Pacific_Islander': '%as.pacif', \
                 'Student_Count_Multi': '%multirace', \
                 'Student_Count_Hawaiian_Pacific_Islander': '%hw.pacif', \
                 'Student_Count_Ethnicity_Not_Available': '%na'}

for count, perc in dmg_perc_cols.items():
    cps_df[perc] = cps_df[count] / cps_df['Student_Count_Total']

### School Types

In [0]:
alt.Chart(cps_df).mark_bar().encode(
    x='Primary_Category:N',
    y='count():Q',
).properties(
    title={
      "text": ["Chicago Public Schools:", "Primary Grade-Level Designations"],
      "subtitle": ["Counts of Elementary, Middle, or High Schools"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

Output hidden; open in https://colab.research.google.com to view.

Most of the schools in the dataset are categorized primarily as Elementary schools. These designations however do not show the complete picture since many schools offer middle school within a larger elementary school, some offer it with high school, and still others offer grade levels from kindergarten all the way to high school. The following graphic demonstrates this.

In [0]:
alt.Chart(cps_df).mark_circle().encode(
    alt.X('Grades_Offered_All'),
    alt.Y('Student_Count_Total', bin=True),
    color='count()',
).properties(
    title={
      "text": ["Chicago Public Schools:", "Grades Offered vs. School Size"],
      "subtitle": ["That's probably Lane Tech at the top"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
alt.Chart(cps_df).mark_point().encode(
    alt.X('Grades_Offered_All'),
    alt.Y('count():Q'),
    size=alt.Size('Student_Count_Total',
                  scale=alt.Scale(domain=[0,25], range=[1,50]),
                  legend=alt.Legend(values=[50, 100, 150]))
).properties(
    title={
      "text": ["Chicago Public Schools:", "Grades Offered vs. School Size"],
      "subtitle": ["That's probably Lane Tech at the top"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

Output hidden; open in https://colab.research.google.com to view.

There are a lot of Pre-Kindergarten to 8th grade schools, but high schools make up the numbers with their student population size.

### Demographics

In [0]:
lowinc_v_white = alt.Chart(cps_df).mark_point().encode(
    x='%low_inc',
    y='%white',
    color='Rating_Status',
    shape='Rating_Status'
).properties(
    title={
      "text": ["Chicago Public Schools:", "% Low-Income vs. % White & School Performance"], 
      "subtitle": ["Majority white schools are probably not majority low-income", "(only two exceptions)"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)
lowinc_v_white

Output hidden; open in https://colab.research.google.com to view.

Each Chicago public school has information about student population demographics. There is also a performance metric ('Rating Status') by which each school's educational attainment is assessed. This graph tells us that most majority-white schools are made up of 50% or less students who come from low-income families. It also tells us that underperforming schools tend to be very poor, and most likely non-white.

In [0]:
lowinc_v_hisp = alt.Chart(cps_df).mark_point().encode(
    x='%low_inc',
    y='%hisp',
    color='Rating_Status',
    shape='Rating_Status'
).properties(
    title={
      "text": ["Chicago Public Schools:", "% Low-Income vs. % Hispanic & School Performance"], 
      "subtitle": ["Majority Hispanic schools are probably majority low-income", "Schools not in good standing tend to be poor"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)
lowinc_v_hisp

Output hidden; open in https://colab.research.google.com to view.

This graph is almost the opposite of the previous. Majority hispanic schools are very likely to be majority low-income. In the very low-income end (near 100%), we can see two clusters--almost 100% or almost 0% Hispanic. In other words, there's a large number of completely poor schools that are either all Hispanic or zero Hispanic. I wondered if these clusters speak to racial segregation in the city of Chicago, and guessed that the latter cluster were made up of high African American population schools. The following graphic explores this idea.

In [0]:
alt.Chart(cps_df).mark_circle().encode(
    x='%low_inc',
    y='%hisp',
    color='%black'
).properties(
    title={
      "text": ["Chicago Public Schools Student Population:", "% Low-Income vs. % Hispanic vs. %Black"], 
      "subtitle": ["Large clusters of very poor schools are either", "all Hispanic or all Black "],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

Output hidden; open in https://colab.research.google.com to view.

This graph more or less confirms the previous hypothesis. It seems like the cluster of very poor schools with close to zero Hispanics are made up of almost all Black students.

## CRIME DATA

### Clean Up/Explore

In [0]:
crime_df = read_csv(CRIMES_18 + ".csv")

In [0]:
crime_df.shape

(267767, 22)

In [0]:
crime_df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [0]:
crime_df.dtypes

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                  int64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

In [0]:
crime_df['Location'].isna().sum()

4347

In [0]:
print(crime_df['Primary Type'].nunique())
print(crime_df['Community Area'].nunique())

32
78


In [0]:
''' ADD/MODIFY ATTRIBUTES '''
# Dates
crime_df['Date'] = pd.to_datetime(crime_df['Date'])
crime_df['Year'] = crime_df['Date'].dt.year
crime_df['Month'] = crime_df['Date'].dt.month

### Crime by Neighborhood

#### Set up new df and explore

In [0]:
# Get a df counting crimes by Type within each Community Area
hood_crimes = crime_df.groupby(['Community Area','Primary Type'], as_index=False)[['Date']].count()

In [0]:
hood_crimes.rename(columns = {'Date':'Crime Count'}, inplace = True)
hood_crimes.head()

,Community Area,Primary Type,Crime Count
0,0.0,BATTERY,1
1,0.0,NARCOTICS,1
2,1.0,ARSON,3
3,1.0,ASSAULT,286
4,1.0,BATTERY,732


In [0]:
# Two crimes with null values for community area
print(hood_crimes.shape)
print(hood_crimes['Crime Count'].sum())
print(crime_df['Community Area'].isna().sum())
print(hood_crimes['Crime Count'].max())

(1851, 3)
267765
2
6144


In [0]:
hood_crimes['Primary Type'].value_counts()

NARCOTICS                            78
BATTERY                              78
DECEPTIVE PRACTICE                   77
ASSAULT                              77
CRIMINAL TRESPASS                    77
OTHER OFFENSE                        77
OFFENSE INVOLVING CHILDREN           77
BURGLARY                             77
THEFT                                77
ROBBERY                              77
MOTOR VEHICLE THEFT                  77
CRIMINAL DAMAGE                      77
PUBLIC PEACE VIOLATION               76
CRIM SEXUAL ASSAULT                  76
WEAPONS VIOLATION                    76
SEX OFFENSE                          75
INTERFERENCE WITH PUBLIC OFFICER     70
HOMICIDE                             68
ARSON                                64
STALKING                             61
INTIMIDATION                         57
LIQUOR LAW VIOLATION                 56
KIDNAPPING                           55
OBSCENITY                            50
CONCEALED CARRY LICENSE VIOLATION    35


#### Graphics

In [0]:
alt.Chart(hood_crimes).mark_area().encode(
    x='Community Area:N',
    y='sum(Crime Count)',
    color='Primary Type:N',
).properties(
    title={
      "text": ["Chicago Crimes by Community Area"],
      "subtitle": ["Some neighborhoods have way more crime than others"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

alt.Chart(...)

The breakdown of crime types are not super informative because there are so many (the Skittle colors are pretty, though). However, we can clearly see how some neighborhoods have way more crime than others. It's also hard to tell what the big blue band is---obscenity or arson? (Would not be surprised if it was the former, but I would be surprised that people report that...)

In [0]:
violent_crimes = ['BATTERY', 'ROBBERY', 'ASSAULT', 'HOMICIDE', 'KIDNAPPING', 'HUMAN TRAFFICKING', 'SEX OFFENSE']
property_crimes = ['THEFT', 'BURGLARY', 'MOTOR VEHICLE THEFT', 'ARSON']

In [0]:
alt.Chart(hood_crimes[hood_crimes['Primary Type'].isin(violent_crimes)]).mark_area().encode(
    x='Community Area:N',
    y='sum(Crime Count)',
    color='Primary Type:N',
).properties(
    title={
      "text": ["Chicago Violent Crimes by Community Area"],
      "subtitle": ["Many battery cases"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

alt.Chart(...)

In [0]:
alt.Chart(hood_crimes[hood_crimes['Primary Type'].isin(property_crimes)]).mark_area().encode(
    x='Community Area:N',
    y='sum(Crime Count)',
    color='Primary Type:N',
).properties(
    title={
      "text": ["Chicago Property Crimes by Community Area"],
      "subtitle": ["Many thefts"],
      "color": "teal",
      "subtitleColor": "purple"
    }
)

alt.Chart(...)

### Bugs to fix
***Altair can't process more than 5K rows?***

In [0]:
#alt.data_transformers.enable('csv')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [0]:
alt.Chart(crime_df).mark_bar().encode(
    x='Community Area:N',
    y='count():Q',
)

In [0]:
url = path_to_data(CRIMES_18 + ".json")
crime_df.to_json(url, orient='records')

In [0]:
chart = alt.Chart(crime_df).mark_bar().encode(
    x='Ward',
    y='count()'
)

In [0]:
chart

alt.Chart(...)

In [0]:
#pprint(chart.to_dict())

Pretty printing has been turned OFF


In [0]:
url

'/content/drive/My Drive/Courses/CAPP 30239 - Data Viz for Policy Analysis/Data/Crimes_-_2018.json'

In [0]:
crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Month
0,11954555,JD119522,2018-07-22 17:00:00,094XX S ASHLAND AVE,1110,DECEPTIVE PRACTICE,BOGUS CHECK,CURRENCY EXCHANGE,False,False,2221,22,21.0,73.0,11,NaN,NaN,2018,01/18/2020 03:46:35 PM,NaN,NaN,NaN,7
1,11955005,JD120023,2018-11-18 00:01:00,041XX N LEAVITT ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,1911,19,47.0,5.0,11,NaN,NaN,2018,01/18/2020 03:46:35 PM,NaN,NaN,NaN,11
2,11676380,JC249699,2018-05-21 00:00:00,115XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,True,False,524,5,34.0,53.0,02,1170399.0,1828251.0,2018,01/18/2020 03:44:11 PM,41.684201,-87.651852,"(41.684200957, -87.651851521)",5
3,11586454,JB556715,2018-12-17 09:18:00,038XX W ADAMS ST,2017,NARCOTICS,MANU/DELIVER:CRACK,SIDEWALK,True,False,1122,11,28.0,26.0,18,NaN,NaN,2018,01/18/2020 03:44:11 PM,NaN,NaN,NaN,12
4,11552383,JB574109,2018-12-31 16:00:00,106XX S OGLESBY AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,434,4,7.0,51.0,02,1194027.0,1835003.0,2018,01/18/2020 03:44:11 PM,41.702184,-87.565137,"(41.70218374, -87.565137272)",12


In [0]:
s = alt.sample(crime_df, n=None, frac=None)

In [0]:
type(s)

pandas.core.frame.DataFrame

In [0]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

## Encodings
- length
- color
- shape
- opacity
- size
- area